In [1]:
# 1. 创建HD钱包，用于批量生成地址。
print("1. 创建HD钱包，用于批量生成地址。")

from web3 import Web3
w3 = Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/5be23a050401499fb951be2e12178e01'))
# test mnemonic from ganache (don't use it!)
mnemonic = "air organ twist rule prison symptom jazz cheap rather dizzy verb glare jeans orbit weapon universe require tired sing casino business anxiety seminar hunt"
w3.eth.account.enable_unaudited_hdwallet_features()


1. 创建HD钱包，用于批量生成地址。


In [3]:
# 2. 利用HD钱包，生成10个钱包地址。
print("2. 利用HD钱包，生成10个钱包地址。")

addresses = []
for i in range(10):
    address = w3.eth.account.from_mnemonic(mnemonic, account_path=f"m/44'/60'/0'/0/{i}")
    addresses.append(address.address)
addresses

2. 利用HD钱包，生成10个钱包地址。


['0x83E5B09c54C4EB904B9bC842Acab9218c2297d6d',
 '0xF44F814ABa3e6BC091487Cf313B49F109550d086',
 '0xC08A302438EaA60adE93196A527A837AA1CA5A3f',
 '0x57171534c9616bB635351deB9d4AA009Fc0d6931',
 '0xFECb70fD6b9414ff7B58C6989D44AFA4a0511D6d',
 '0x897366fBfD8505dE0D772e2F34CF99ac692a9B15',
 '0x5412DD89dD6B707fA816a3b8E0BDFe44A46CA152',
 '0x9c85ee2fFB694A161b59D697ca560Aa2e1a98E6E',
 '0xaBEC7899686e8FE4658bcce8391Fb4e3A70C8868',
 '0x7610CfA2931e9D36CD1aF96599a5ed7886561147']

In [8]:
# 3. 创建wallet，发送代币用。
print("3. 创建wallet，发送代币用。")

private_key = "0x21ac72b6ce19661adf31ef0d2bf8c3fcad003deee3dc1a1a64f5fa3d6b049c06"
account = w3.eth.account.from_key(private_key)
account.address


3. 创建wallet，发送代币用。


'0x338f8891D6BdC58eEB4754352459cC461EfD2a5E'

In [5]:
# 4. 创建Airdrop合约。
print("4. 创建Airdrop合约。")

# 合约 abi
airdrop_abi = '[{"inputs":[{"internalType":"uint256[]","name":"_arr","type":"uint256[]"}],"name":"getSum","outputs":[{"internalType":"uint256","name":"sum","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address payable[]","name":"_addresses","type":"address[]"},{"internalType":"uint256[]","name":"_amounts","type":"uint256[]"}],"name":"multiTransferETH","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"_token","type":"address"},{"internalType":"address[]","name":"_addresses","type":"address[]"},{"internalType":"uint256[]","name":"_amounts","type":"uint256[]"}],"name":"multiTransferToken","outputs":[],"stateMutability":"nonpayable","type":"function"}]'

# 合约地址 测试网
airdrop_address = '0x71C2aD976210264ff0468d43b198FD69772A25fa'

# 创建合约实例
airdrop_contract = w3.eth.contract(address=airdrop_address, abi=airdrop_abi)
airdrop_contract

4. 创建Airdrop合约。


In [6]:
# 5. 读取一个地址的ETH余额。
print("5. 读取一个地址的ETH余额。")

# 目标地址
target_address = addresses[8]

# 读取 ETH 余额
balanceETH = w3.eth.get_balance(target_address)

print(balanceETH)

5. 读取一个地址的ETH余额。
200000000000000


In [7]:
# 6. 调用multiTransferETH()函数，给每个钱包转 0.0001 ETH
print("6. 调用multiTransferETH()函数，给每个钱包转 0.0001 ETH")

# 发起交易
amounts = (len(addresses)) * [w3.to_wei(0.0001, 'ether')]
# 调用方法
tx = airdrop_contract.functions.multiTransferETH(addresses, amounts).build_transaction({
    'from': account.address,
    'value': w3.to_wei(0.001, 'ether'),
    'gas': 2000000,  # 根据需要调整
    'nonce': w3.eth.get_transaction_count(account.address)
})

# 发起交易
signed_tx = w3.eth.account.sign_transaction(tx, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

# 等待交易上链
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

balanceETH2 = w3.eth.get_balance(addresses[8])
print("转账后余额：", balanceETH2)

6. 调用multiTransferETH()函数，给每个钱包转 0.0001 ETH
转账后余额： 300000000000000
